In [ ]:
# Install necessary libraries
! pip install datasets
! pip install huggingface_hub
# Connect to hugging face and wandb
! huggingface-cli login --token YOUR_HF_TOKEN

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import load_dataset

In [ ]:
#dataset_1 = load_dataset("esg-x/esg-triplet-dataset", token=True)
dataset_2 = load_dataset("esg-x/esg-mostly-subsuming-and-overlapping-dataset", token=True)
dataset_3 = load_dataset("esg-x/esg-mostly-adjacent-dataset", token=True)

README.md:   0%|          | 0.00/659 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/49.0M [00:00<?, ?B/s]

val-00000-of-00001.parquet:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26085 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/8695 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8696 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['pairs_unique_id', 'doc_id', 'anchor', 'positive', 'negative'],
        num_rows: 26085
    })
    val: Dataset({
        features: ['pairs_unique_id', 'doc_id', 'anchor', 'positive', 'negative'],
        num_rows: 8695
    })
    test: Dataset({
        features: ['pairs_unique_id', 'doc_id', 'anchor', 'positive', 'negative'],
        num_rows: 8696
    })
})

In [ ]:
from datasets import concatenate_datasets
# Concatenate the datasets
merged_dataset = concatenate_datasets([dataset_2['train'], dataset_3['train']])
merged_val_dataset = concatenate_datasets([dataset_2['val'], dataset_3['val']])
merged_test_dataset = concatenate_datasets([dataset_2['test'], dataset_3['test']])


In [ ]:
merged_dataset = merged_dataset.shuffle(seed=42)
merged_val_dataset = merged_val_dataset.shuffle(seed=42)
merged_test_dataset = merged_test_dataset.shuffle(seed=42)

In [ ]:
from datasets import DatasetDict

In [ ]:
dataset = DatasetDict({
    'train': merged_dataset,
    'val': merged_val_dataset,  # This is the validation set
    'test': merged_test_dataset
})

In [ ]:
dataset = dataset.remove_columns("pairs_unique_id")

In [ ]:
!git config --global credential.helper store

In [ ]:
dataset.push_to_hub("esg-x/triplet-dataset", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/53 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/18 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/18 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/esg-x/triplet-dataset/commit/37d8ace2aad4d21811f8029db79fc121816e8557', commit_message='Upload dataset', commit_description='', oid='37d8ace2aad4d21811f8029db79fc121816e8557', pr_url=None, pr_revision=None, pr_num=None)

# hard negatives

In [ ]:
dataset = load_dataset("esg-x/triplet-dataset", token=True)

README.md:   0%|          | 0.00/598 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/98.0M [00:00<?, ?B/s]

val-00000-of-00001.parquet:   0%|          | 0.00/32.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/32.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52170 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/17390 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/17392 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.remove_columns("negative")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['doc_id', 'anchor', 'positive'],
        num_rows: 52170
    })
    val: Dataset({
        features: ['doc_id', 'anchor', 'positive'],
        num_rows: 17390
    })
    test: Dataset({
        features: ['doc_id', 'anchor', 'positive'],
        num_rows: 17392
    })
})

In [ ]:
import random  # Import the random module to sample negative anchors

# Function to create a hard negative sample
def create_hard_negatives(example, grouped_by_doc_id):
    current_anchor = example['anchor']
    doc_id = example['doc_id']

    # Get a different anchor from the same doc_id
    candidates = [pair[0] for pair in grouped_by_doc_id[doc_id] if pair[0] != current_anchor]

    negative_anchor = random.choice(candidates)

    # Return the example with the new "negative" column
    return {
        'doc_id': example['doc_id'],
        'anchor': example['anchor'],
        'positive': example['positive'],
        'negative': negative_anchor
    }

# Function to process the entire dataset
def process_dataset(dataset):
    # First, group by doc_id to ensure we get negatives from the same doc_id
    grouped_by_doc_id = {}

    # Create a dictionary where the key is doc_id and the value is a list of (anchor, positive) pairs
    for example in dataset:
        doc_id = example['doc_id']
        if doc_id not in grouped_by_doc_id:
            grouped_by_doc_id[doc_id] = []
        grouped_by_doc_id[doc_id].append((example['anchor'], example['positive']))

    # Apply the function to each example in the dataset
    return dataset.map(lambda x: create_hard_negatives(x, grouped_by_doc_id))



In [ ]:
# Apply this function to train, val, and test sets
dataset['train'] = process_dataset(dataset['train'])
dataset['val'] = process_dataset(dataset['val'])
dataset['test'] = process_dataset(dataset['test'])


Map:   0%|          | 0/52170 [00:00<?, ? examples/s]

Map:   0%|          | 0/17390 [00:00<?, ? examples/s]

Map:   0%|          | 0/17392 [00:00<?, ? examples/s]

{'doc_id': Value(dtype='int64', id=None), 'anchor': Value(dtype='string', id=None), 'positive': Value(dtype='string', id=None), 'negative': Value(dtype='string', id=None)}


In [ ]:
# First, group by doc_id to ensure we get negatives from the same doc_id
grouped_by_doc_id = {}

# Create a dictionary where the key is doc_id and the value is a list of (anchor, positive) pairs
for example in dataset['test']:
    doc_id = example['doc_id']
    if doc_id not in grouped_by_doc_id:
        grouped_by_doc_id[doc_id] = []
    grouped_by_doc_id[doc_id].append((example['anchor'], example['positive']))